<h3> Topographic Classification: a Machine Learning Approach using Support Vector Machines, Noteboook II</h3>

In this notebook we continue experiments with Support Vector Machine based models for identifying and mapping topographic landscapes. We continue with the same Eastern U.S. geography that was at the center of prior study. 

Preliminary results in applying the SVM to differentiate and classify the coastal plain from the upland are encouraging with classification accuracies exceeding 90%. 

The overarching assumption here continues to be that the coastal plain province, and by extension coastal plains in general, present topographic characteristics that differ geomorphically/geomorphometrically with the adjacent upland. Further, these differientiating characteristics can measured and used to build  classification model that distinguishes in some meaningful way, present a less topographically varied (i.e. rough) character than do the interior uplands [onto which the coastal plains abutt). Further, we assume that, in the presence of true assumptions, we can further identify, quantitatively model, and go on to map this difference and the two provinces--reliably. Repeatably. Maybe even globally!

**NOTE2 THESE DATA ARE _NOT_ SLOPE CONSTRAINED: IN THIS NOTEBOOK WE ARE USING ALL SLOPES RANGING FROM 0 TO (up to) 90 DEGREES (if they exist)**

**Load Requisite Modules, Libraries, and Magics:**

In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score   # not using
from sklearn.model_selection import KFold             # not using
from sklearn.model_selection import validation_curve  # not using
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline            # not using
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import seaborn as sns

import sqlite3

sns.set()
%matplotlib inline

**Load the [labeled] sample observation data**

In [2]:
datapath='/Users/paulp/GoogleDrive/projects/GRASSData/USAtl_CoastalPlain_Prototype_LCC/user/sqlite/'
file='sqlite.db'

conn=sqlite3.connect(datapath+file)   # create a connection to the GRASS database file:

# load the contents of the vector point file: train_test_10k_pts into a pandas dataframe
# note that we are loading only those records whose elevation (z) is > 0 meters.
# note that we are also dropping all records that contain one or more missing (NaN) values.
df = pd.read_sql_query('select * from train_test_10k_pts where z > 0.0', conn).dropna()

# build the standardized X features and y target arrays:
scaler = StandardScaler()
X=scaler.fit_transform(np.array(df[['z','slope','distance']]) )
y=np.array(df['label']) 
print(np.shape(y), np.shape(X))

(9787,) (9787, 3)


In [ ]:
df.describe()
#df.isnull().values.any()
#df.isnull().sum()

**Plot the Sample Data--Premodeling (Elevation, Slope, Distance to Shore)**

In [ ]:
### Plots for Elevation vs Slope

fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(14,7)) 

ax1.scatter(X[:,0],X[:,1], c=y, s=14, cmap=mpl.cm.Paired)
ax1.set_xlabel('Elevation (m)')
ax1.set_ylabel('Slope (degrees)')
ax1.set_title('Elevation vs. Slope')

ax2.scatter(X[:,0],X[:,1], c=y, s=14, cmap=mpl.cm.Paired)
ax2.set_xlim(-1.25, 0)
ax2.set_ylim(-0.25, 0.25)
ax2.set_xlabel('Elevation (m)')
ax2.set_ylabel('Slope (degrees)')
ax2.set_title('Elevation vs. Slope')

In [ ]:
### Plots for Elevation vs Distance to the Ocean Shore

fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(14,7)) 

ax1.scatter(X[:,0],X[:,2], c=y, s=14, cmap=mpl.cm.Paired)
ax1.set_xlabel('Elevation (m)')
ax1.set_ylabel('Distance from the Ocean Shore (m)')
ax1.set_title('Elevation vs. Distance to Shore')

ax2.scatter(X[:,0],X[:,2], c=y, s=14, cmap=mpl.cm.Paired)
ax2.set_xlim(-1.25, 0)
ax2.set_ylim(-1.4, 0)
ax2.set_xlabel('Elevation (m)')
ax2.set_ylabel('Distance from the Ocean Shore (m)')
ax2.set_title('Elevation vs. Distance to Shore')

In [ ]:
### Plots for Slope vs Distance to the Ocean Shore

fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(14,7)) 

ax1.scatter(X[:,1],X[:,2], c=y, s=14, cmap=mpl.cm.Paired)
ax1.set_xlabel('Slope (degrees)')
ax1.set_ylabel('Distance from the Ocean Shore (m)')
ax1.set_title('Slope vs. Distance to Shore')

ax2.scatter(X[:,1],X[:,2], c=y, s=14, cmap=mpl.cm.Paired)
ax2.set_xlim(-0.5, 14)
ax2.set_ylim(-1.3, 0)
ax2.set_xlabel('Slope (degrees)')
ax2.set_ylabel('Distance from the Ocean Shore (m)')
ax2.set_title('Slope vs. Distance to Shore')

**Create the training and testing data subsets**

The training data is used to fit the Support Vector Machine (SVM) model (the SVM classifer is actually fit to the training data). We do initial testing and tuning of the model via cross-validation (coming next) and then "test" the model against the test data subset for generalization--its ability to predict using datasets of unknown classification.


The holdout split of the test data is typically 20% (Aurelien, Geron, 2017. Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems. O-Reilly Media, Inc. Per Andrew Ng (Stanford U. Machine Learning Course): training 60%, validation 20%, testing 20%. Since we're doing kfold cross-validation
we don't need the validation set, so we'll stick with the 'Pareto' 80/20
rule! However...cross-validation and optimization on approximately 8000 sample points is sure to chew up lots of computing resources (O(N^3) actually) and so require a lot of time (perhaps days) to process. To get around this issue to some degree we will split the training data (Xtrain) in half and do the cross-validation and optimization on those fewer points. Once optimized hyper-parameters are in hand we can then train the SVC using those parameters against the full training set. 

In [3]:
holdout=0.2
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=holdout, random_state=42)

print('Total Sample Observations:', len(X))
print('Observations in training set:', len(Xtrain))
print('Observations in test set:', len(Xtest))

Total Sample Observations: 9787
Observations in training set: 7829
Observations in test set: 1958


**Cross-Validation and Parameter Tuning to Optimize the SVC**

We use cross-validation in order to identify optimal values for the hyperparameters that will define our SVM. The parameters of interest are: kernel (linear, polynmial, or radial basis), gamma (if a radial basis function kernel proves best), degree (if a polynomial kernel is the best fit), and C the tolerance for data points falling inside the hyperplane margins.

Note: some of the reporting code was copied, with modification, from the Sci-kit learn GridSearchCV documentation: http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html

In [8]:
%%time
print('Using', len(Xtrain), 'sample points for cross-validation')
#parameters=[{'kernel':['rbf'], 'C':[1,10,100,1000], 'gamma':[7,8,9,10,11,12,14]}]

# parameters for the initial "wide-grid" search:
parameters=[{'kernel':['rbf'], 'C':[0.1,1.0,10,100,1000], 'gamma':[0.002,0.02,0.2,2,20,200,2000]}]

#parameters=[{'kernel':['rbf'], 'C':[0.1,1.0,10,100,1000], 'gamma':[0.002,0.02,0.2,2,20]}]


scores=['precision','recall']
for score in scores:
    print('Optimizing the SVC for %s' % score)
    cv = GridSearchCV(estimator=svm.SVC(C=1),param_grid=parameters, cv=5, n_jobs=-1, scoring='%s_macro' % score)
    cv.fit(Xtrain,ytrain)
    print("Best parameters set found on development set:")
    print()
    print(cv.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = cv.cv_results_['mean_test_score']
    stds = cv.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, cv.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = ytest, cv.predict(Xtest)
    print(classification_report(y_true, y_pred))
    print()

Using 7829 sample points for cross-validation
Optimizing the SVC for precision
Best parameters set found on development set:

{'C': 0.1, 'gamma': 200, 'kernel': 'rbf'}

Grid scores on development set:

0.841 (+/-0.098) for {'C': 0.1, 'gamma': 0.002, 'kernel': 'rbf'}
0.877 (+/-0.018) for {'C': 0.1, 'gamma': 0.02, 'kernel': 'rbf'}
0.908 (+/-0.018) for {'C': 0.1, 'gamma': 0.2, 'kernel': 'rbf'}
0.926 (+/-0.005) for {'C': 0.1, 'gamma': 2, 'kernel': 'rbf'}
0.935 (+/-0.007) for {'C': 0.1, 'gamma': 20, 'kernel': 'rbf'}
0.944 (+/-0.014) for {'C': 0.1, 'gamma': 200, 'kernel': 'rbf'}
0.916 (+/-0.008) for {'C': 0.1, 'gamma': 2000, 'kernel': 'rbf'}
0.877 (+/-0.019) for {'C': 1.0, 'gamma': 0.002, 'kernel': 'rbf'}
0.903 (+/-0.009) for {'C': 1.0, 'gamma': 0.02, 'kernel': 'rbf'}
0.926 (+/-0.007) for {'C': 1.0, 'gamma': 0.2, 'kernel': 'rbf'}
0.935 (+/-0.016) for {'C': 1.0, 'gamma': 2, 'kernel': 'rbf'}
0.939 (+/-0.011) for {'C': 1.0, 'gamma': 20, 'kernel': 'rbf'}
0.944 (+/-0.011) for {'C': 1.0, 'gamma': 

**Create a Confusion Matrix**

Display a count of the number of observations correctly classified and number nor correctly classified for each class label

In [ ]:
# create a confusion matrix to get a count of the number of observations correctly classified
# and the number NOT correctly classified:
#print(confusion_matrix(ytest,sv.predict(Xtest) ))
print('Confusion Matrix:')
print(pd.DataFrame(confusion_matrix(y_true,y_pred), index=[-1,1], columns=[-1,1]) )
print()
print(cv.best_estimator_.score(Xtest, ytest).round(3))
print()
print('Geometric Mean:' )
print( )

**Generate (learn) an SVM model**

Using the best estimators from the GridSearch cross-validation build a new (optimized) SVC with the training data set.

In [ ]:
%%time
sv=svm.SVC(C=100, kernel='rbf', gamma=12).fit(Xtrain,ytrain)
print('Number of support vectors (class -1 class 1):', sv.n_support_)
print('Model score:', sv.score(Xtrain,ytrain))

**Classify (predict) the U.S. Atlantic Coast test data set**

In [ ]:
y_pred = sv.predict(Xtest)
print(y_pred)
np.shape(y_pred)

**Test the model against a fresh data set**

We generated another 7000 point random sample across the study area in GRASS in order to further test the efficacy of the new sv SVC model.

In [ ]:
# load the new sample point set from GRASS, selecting only those records
# whose elevation value (value) is >= 0 meters:

df7 = pd.read_sql_query('select * from test_7k_pts where value >= 0', conn)

# scale the data to standard normal with mean=0 and standard deviation=1:
X7 = scaler.fit_transform(np.array(df7[['value','slope','distance']]))
print('Fitting', len(X7), 'points to sv')
y7 = sv.predict(X7)


In [ ]:
print(y7)
print(np.shape(df1), np.shape(X7), np.shape(y7))
df_ = df7.join(pd.DataFrame(y7, columns=['label']))
df7
df_.to_csv('/Users/paulp/GoogleDrive/projects/CoastalPlains/data/fit_7k_pts', index=False)

**Test the model in a location outside the U.S. Atlantic Coast domain**

We've downloaded a small section of the South American Coast in the northeast across the Guyana and Suriname Republics. These are areas where there is a well-defined, albeit small, coastal plain bounded by mesozoic and older rock bodies. We'll run a set of 5000 sample points thru the sv SVC to see how well a model, contoured to fit the geomorphology of the east coast of North America generalizes to other areas. Ultimately, in order to present a more globally generalized model--that is, one that 'best-fits' coastal regions and coastal plains (where they exist) around the world. Tall order perhaps, indeed! 

In [ ]:
# load the new sample point set from GRASS, selecting only those records
# whose elevation value (value) is >= 0 meters:

dfbc = pd.read_sql_query('select * from Guyanas_5kpts where value >= 0', conn).dropna()

loc = np.array(dfbc[['cat','x','y']])
# scale the data to standard normal with mean=0 and standard deviation=1:
X = scaler.fit_transform(np.array(dfbc[['elevation','slope','distance']]))
print('Fitting', len(X), 'points to sv')
y = sv.predict(X)
print('Done')

In [ ]:
print(np.shape(loc))
print(np.shape(X))
print(np.shape(y))

In [ ]:
dfg1 = pd.DataFrame(loc, columns=['cat','x','y'])
dfg2 = pd.DataFrame(X, columns=['elevation','slope','distance'])
dfg3 = pd.DataFrame(y, columns=['label'])

dfg = pd.concat([dfg1,dfg2,dfg3], axis=1)
dfg.describe()

In [ ]:
dfg.to_csv('/Users/paulp/GoogleDrive/projects/CoastalPlains/data/guyana_5k_pts.csv', index=False)

Results for the Guyanas data were plotted in QGIS. Upon inspection it was immediately clear that the North American-based SVM was far from optimal for the northeastern corner of South America. The problem, I'm quite certain, is due in the largest part to model overfitting. I've 'tuned' the N. American SVM to closely to the nuances of that region's topography such that the model, in its present form, generalizes poorly. This was, believe it or not, not unexpected. 

The fix is to 'detune' the model for N. America and retune using more sophisticated optimizations methods, coupled with training data taken from a broader geographic region. In the future, we will experiment with fitting a new SVM classifier using:

-  gradient descent and/or
-  stochastic gradient descent 

to isolate the optimal hyper-parameters and learning rates, all with the goal of crafting a coastal model that 'learns on the fly'. That is, as new data are added for model training, the model can, without rebuilding, adapt to new data, and hence new and differing topographic characteristics for the world's coastal regions. In the end we'll have a dynamic, adptive learning model that can discriminate and map the coastal plain regions, and ultimately, the entirety of the continental margin, of which, coastal plains are an integral component. 

Borrowing from a professor of old, Dr. R. K. Bambach: 'The modeling ain't done yet!'

Stay tuned...